# The first section deals with just the magic items and their owners

In [1]:
import pandas as pd
import openpyxl
import re

#load excel file without formulas
file_path = 'MagicItemList.xlsx'
workbook = openpyxl.load_workbook(file_path, data_only=True)

#specific sheet
sheet_name = 'Items_By_Rarity'
sheet = workbook[sheet_name]

#create empty df with column headers
columns = ['owner', 'item_name', 'spec_prop', 'rarity', 'carried', 'trade_will', 'reward', 'certed']
df = pd.DataFrame(columns=columns)

#iterate through columns, starting from the third row, second column
for col_index in range(2, sheet.max_column + 1):
    owner = sheet.cell(row=3, column=col_index).value
    for row_index in range(4, sheet.max_row + 1):
        item_value = sheet.cell(row=row_index, column=col_index).value
        spec_prop = ""  #initialize spec_prop empty string to avoid NaNs
        if pd.notna(item_value): 
            text = str(item_value)
            match = re.search(r'\((.*?)\)', text)
            if match:
                spec_prop = match.group(1)
                item_name = text.replace(f'({spec_prop})', '').strip() #strip the special properties inside ()
            else:
                item_name = text #item name without ()
            
            #determine item rarity from the color of the text
            font_color = sheet.cell(row=row_index, column=col_index).font.color
            rarity = None
            if font_color.rgb == "FFFFC000":  #Orange
                rarity = "Unique"
            elif font_color.rgb == "FF7030A0":  #Purple
                rarity = "Legendary"
            elif font_color.rgb == "FF0070C0":  #Blue
                rarity = "Very Rare"
            elif font_color.rgb == "FF00B050":  #Green
                rarity = "Rare"
            elif font_color.rgb == "FFFF0000":  #Red
                rarity = "Uncommon"
            else: 
                rarity = "Common"  #Black
            
            #determine the trade willingness of the item from the color of the cell fill
            fill_color = sheet.cell(row=row_index, column=col_index).fill.start_color.index
            trade_will = None
            if fill_color == 2:  #Light Grey, Background 2, Darker 25%
                trade_will = "Untradeable"
            elif fill_color == 5:  #Orange, Accent 2, Lighter 60%
                trade_will = "Red"
            elif fill_color == 7:  #Gold, Accent 4, Lighter 60%
                trade_will = "Amber"
            elif fill_color == 9:  #Green, Accent 6, Lighter 60%
                trade_will = "Green"
            else: 
                trade_will = "FAILED" #check to confirm all items had a fill color and no random cells
                print(f"Fill color RGB value: {fill_color}")
            
            #determine if the item is actively being used (carried) by a character
            borders = sheet.cell(row=row_index, column=col_index).border
            carried = borders and borders.top.style == "thin" and borders.bottom.style == "thin" and borders.left.style == "thin" and borders.right.style == "thin"
            
            #determine if the item was a DM or service reward if the text is italics 
            italic = sheet.cell(row=row_index, column=col_index).font.italic
            reward = italic
            
            #determine if the item has a physical cert if the text is bold
            bold = sheet.cell(row=row_index, column=col_index).font.bold
            certed = bold
            
            #popluate the item row
            df.loc[len(df)] = [owner, item_name, spec_prop, rarity, carried, trade_will, reward, certed]

#save df to csv file to review
csv_file_path = 'output.csv'
df.to_csv(csv_file_path, index=False)

print("DataFrame has been saved to:", csv_file_path)

DataFrame has been saved to: output.csv


In [2]:
#define function that takes dataframe as input, checks that df for the book names and returns count of those that can be used
def book_count(dataframe):
    
    #define book names list
    book_names = [
        'Manual of Gainful Exercise', #STR tome
        'Manual of Quickness of Action', #DEX tome
        'Manual of Bodily Health', #CON tome
        'Tome of Clear Thought', #INT tome
        'Tome of Understanding', #WIS tome
        'Tome of Leadership and Influence' #CHA tome
    ]
    
    #initialize empty df for function return 
    book_counts = {}
    
    #for loop that uses the new df to count the books that can be used
    for book_name in book_names:
        count = dataframe[(dataframe['item_name'].str.count(book_name) > 0) & (dataframe['trade_will'] != 'Untradeable')].shape[0]
        book_counts[book_name] = count
        print(f"{book_name}: {count}") #display the count for that book
    
    return book_counts

result = book_count(df)

Manual of Gainful Exercise: 5
Manual of Quickness of Action: 7
Manual of Bodily Health: 2
Tome of Clear Thought: 7
Tome of Understanding: 6
Tome of Leadership and Influence: 8


In [2]:
#function that displays a list of items for a specific character that the user can select from a menu of options
def list_owners_and_items(dataframe):
    
    #create a list of characters with duplicates
    unique_owners = dataframe['owner'].unique()
    
    #print list of characters with numbers for easy selection
    for i, owner in enumerate(unique_owners, start=1):
        print(f"{i}. {owner}")
    
    #prompt for character choice with character name or number
    while True:
        try:
            choice = input("Please enter the character name or corresponding number: ")
            if choice.isdigit():
                choice = int(choice)
                if 1 <= choice <= len(unique_owners):
                    owner = unique_owners[choice - 1]
                    break
            elif choice in unique_owners:
                owner = choice
                break
            print("Invalid input. Please enter a valid character name or number.")
        except KeyboardInterrupt:
            print("\nOperation aborted.")
            return
    
    #filter new dataframe for specific character
    owner_items = dataframe[dataframe['owner'] == owner]
    
    #count total magic items and carried magic items
    total_items = len(owner_items)
    total_carried_items = owner_items['carried'].sum()
    
    #count items by rarity
    items_by_rarity = owner_items['rarity'].value_counts()
     
    print("\nItems for", owner)
    print(f"Total items: {total_items}")
    print(f"Total carried items: {total_carried_items}")
    print("Items by rarity:")
    for rarity, count in items_by_rarity.items():
        print(f"{rarity}: {count}")
    print("\nItem List")
    
    #display magic items for selected character appending the item's spec_prop in () and an * if it is being carried
    for idx, row in owner_items.iterrows():
        item_name = row['item_name']
        spec_prop = row['spec_prop']
        carried = '*' if row['carried'] else ''
        spec_prop_text = f" ({spec_prop})" if spec_prop else ''
        print(f"{item_name}{spec_prop_text} {carried}")

list_owners_and_items(df)

1. LEE-ROY!
2. Lynus
3. Zalyria
4. Yensid
5. Superb
6. Sound of Breaking Wood
7. Mi'ttens
8. Roon
9. Brakk
10. Pyro
11. Omni
12. Grinkarr
13. Bryanna
14. Dayne
15. Queequeg
16. Vith
17. Darkness
18. Harlund
19. Eisner
20. Elwood
21. Baldain
22. Zebulon
23. Eitri
24. Eir
25. Cogburn
26. Ordrick
27. Draft
28. Zargon
29. Arryn
30. Jerrica
31. Claudius
32. Glovil
33. Grimmir
34. Azkadellia
35. Flavoris Flavoli
36. Billy
37. Aeric Thunderfoot
38. Mapletalon
39. Quelyn
40. Nanor
41. Cypher
42. Titus
43. Xyn
44. Karatos
45. Nimbus
46. Ziggy
47. Mu-289
48. Hakka
49. Glurp
50. T. Goatwalker T.
51. Roland
52. Trisstina
53. Roscoe
54. Gazkin
55. Riesgraf
56. Ludo
57. Ookgra
58. Throgs
59. Billetera de la Prisión
60. Mula de Articulo
61. Fangelsi Veski
Please enter the character name or corresponding number: 54

Items for Gazkin
Total items: 10
Total carried items: 3
Items by rarity:
Rare: 4
Uncommon: 4
Very Rare: 1
Common: 1

Item List
Manual of Quickness of Action 
Dragonhide Belt +2 
Quartersta

In [4]:
#function that generates a trade list based on magic item rarity and the willingness to trade the item
def generate_trade_list(dataframe):
    
    #map rarity to visually appealing text
    rarity_mapping = {
        'common': 'Common',
        'uncommon': 'Uncommon',
        'rare': 'Rare',
        'very rare': 'Very Rare',
        'legendary': 'Legendary'
    }
    
    #map rarity to visually appealing text and so that red and amber include the lower trade willingness
    trade_will_mapping = {
        'red': ['Red', 'Amber', 'Green'],
        'amber': ['Amber', 'Green'],
        'green': ['Green']
    }
    
    #function inside function that asks the user for the rarity and trade willingness for list
    def prompt_for_option(options, prompt):
        while True:
            input_str = input(prompt)
            if input_str.isdigit():
                option_num = int(input_str)
                if 1 <= option_num <= len(options):
                    return options[option_num - 1]
            elif input_str.lower() in options:
                return input_str.lower()
            print("Invalid input. Please enter a valid option.")
    
    #prompt for rarity
    rarity_options = list(rarity_mapping.keys())
    rarity_prompt = f"Enter the desired rarity ({', '.join(rarity_options)} or 1-{len(rarity_options)}): "
    rarity_input = prompt_for_option(rarity_options, rarity_prompt)
    rarity = rarity_mapping[rarity_input]
    
    #prompt for trade willingness
    trade_will_options = list(trade_will_mapping.keys())
    trade_will_prompt = f"Enter the desired trade willingness ({', '.join(trade_will_options)} or 1-{len(trade_will_options)}): "
    trade_will_input = prompt_for_option(trade_will_options, trade_will_prompt)
    trade_will = trade_will_mapping[trade_will_input]
    
    #filter dataframe on rarity and trade willingness
    filtered_items = dataframe[
        (dataframe['rarity'] == rarity) & (dataframe['trade_will'].isin(trade_will))
    ]
    
    #create set for storing unique item names
    unique_items = set()
    
    #iterate filtered items adding item names to the set
    for idx, row in filtered_items.iterrows():
        item_name = row['item_name']
        spec_prop = row['spec_prop']
        item = f"{item_name} ({spec_prop})" if spec_prop else item_name
        unique_items.add(item)
    
    #create alphabetically sorted paragraph style list
    if unique_items:
        sorted_trade_list = sorted(unique_items)
        trade_list_paragraph = ', '.join(sorted_trade_list)
        return f"{rarity} Trade Items ({', '.join(trade_will)}):\n{trade_list_paragraph}"
    else:
        return f"No {rarity} trade items found for {', '.join(trade_will)}."

#call function with dataframe
result = generate_trade_list(df)
print(result)

Enter the desired rarity (common, uncommon, rare, very rare, legendary or 1-5): 4
Enter the desired trade willingness (red, amber, green or 1-3): 2
Very Rare Trade Items (Amber, Green):
Abracadabrus, Absorbing Tattoo (Acid, Harmonious), Absorbing Tattoo (Fire), Absorbing Tattoo (Force), Absorbing Tattoo (Force, War Leader), Absorbing Tattoo (Lightning, Guardian), Absorbing Tattoo (Necrotic), Absorbing Tattoo (Psychic), Amethyst Lodestone, Amethyst Lodestone (Compass), Amulet of the Planes, Amulet of the Planes (Naerth's Planar Compass), Animated Shield, Anstruth Harp, Anstruth Harp (Grig Fiddle), Armor of Safeguarding, Bag of Devouring, Baleful Talon, Battleaxe +3 (Skeggox), Belt of Fire Giant Strength, Belt of Stone Giant Strength, Blast Scepter, Blast Scepter , Bobbing Lily Pad, Breastplate +2, Carpet of Flying (3x5), Carpet of Flying (4x6), Carpet of Flying (5x7), Carpet of Flying (6x9), Cauldron of Rebirth, Cloak of Arachnida, Cloak of arachnida, Crystalline Chronicle, Dancing Swor

In [7]:
def generate_rarity_summary_table(dataframe):
    #calculate total items of each rarity
    total_rarity = dataframe['rarity'].value_counts().sort_index()
    
    #calculate total carried items of each rarity
    total_carried = dataframe[dataframe['carried']]['rarity'].value_counts().sort_index()
    
    #calculate total untradeable items of each rarity
    total_untradeable = dataframe[dataframe['trade_will'] == 'Untradeable']['rarity'].value_counts().sort_index()
    
    #create summary table
    summary_table = pd.DataFrame({
        'Total': total_rarity,
        'Red': dataframe[dataframe['trade_will'] == 'Red']['rarity'].value_counts().sort_index(),
        'Amber': dataframe[dataframe['trade_will'] == 'Amber']['rarity'].value_counts().sort_index(),
        'Green': dataframe[dataframe['trade_will'] == 'Green']['rarity'].value_counts().sort_index(),
        'Untradeable': total_untradeable,
        'Carried': total_carried,
    }).fillna(0).astype(int)
    
    #add sum row
    summary_table.loc['Sum'] = [
        total_rarity.sum(),
        summary_table['Red'].sum(),
        summary_table['Amber'].sum(),
        summary_table['Green'].sum(),
        summary_table['Untradeable'].sum(),
        total_carried.sum()
    ]
    
    return summary_table

#call function with dataframe
summary_table = generate_rarity_summary_table(df)
print(summary_table)

           Total  Red  Amber  Green  Untradeable  Carried
Common       144   48     20     76            0       28
Legendary    142   64     51     27            0       39
Rare         536  101    214    221            0       75
Uncommon     315   57     87    170            1       52
Unique        10    0      0      0           10        3
Very Rare    421  104    181     78           58      130
Sum         1568  374    553    572           69      327


# This next section deals with the characters and their level

In [8]:
#load excel file without formulas
file_path = 'MagicItemList.xlsx'
workbook = openpyxl.load_workbook(file_path, data_only=True)

#specific sheet
sheet_name = 'Classes'
sheet = workbook[sheet_name]

#create second df with class and level information
class_df = pd.read_excel(file_path)

#assign first row as column names
class_df.columns = class_df.iloc[0]  
class_df = class_df[1:] 

class_df.head(10)

,Campaign,Season,WPM,Red War,Name,Subclass,Artificer,Barbarian,Bard,Cleric,...,Rogue,Sorcerer,Warlock,Wizard,Level,of Classes,NaN,NaN,NaN,NaN
1,Forgotten Realms,5,HT,NaN,LEE-ROY!,Path of the Totem,NaN,20,NaN,NaN,...,NaN,NaN,NaN,NaN,20,1,NaN,Total,67.000000,NaN
2,Forgotten Realms,1,NaN,Y,Lynus,College of Lore,NaN,NaN,19,1,...,NaN,NaN,NaN,NaN,20,2,NaN,NaN,NaN,NaN
3,Forgotten Realms,2,HT T4,NaN,Zalyria,Assassin / Champion,NaN,NaN,NaN,NaN,...,15,NaN,NaN,NaN,20,2,NaN,T4,41.000000,0.611940
4,Forgotten Realms,2,NaN,Y,Yensid,School of Evocation,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,20,20,1,NaN,T3,11.000000,0.164179
5,Forgotten Realms,5,HT T3,NaN,Superb,Great Old One,NaN,NaN,NaN,NaN,...,NaN,NaN,20,NaN,20,1,NaN,T2,10.000000,0.149254
6,Forgotten Realms,7,Y,NaN,SoBW,Way of the Open Hand,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,20,2,NaN,T1,5.000000,0.074627
7,Forgotten Realms,7,NaN,NaN,Mi'ttens,Hunter,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,20,1,NaN,NaN,NaN,NaN
8,Forgotten Realms,4,Y,Y,Brakk,Life Domain,NaN,NaN,NaN,20,...,NaN,NaN,NaN,NaN,20,1,NaN,56,0.835821,NaN
9,Forgotten Realms,6,HT,Y,Pyro,Draconic,NaN,NaN,NaN,NaN,...,NaN,20,NaN,NaN,20,1,NaN,NaN,NaN,NaN
10,Forgotten Realms,8,Y,NaN,Ordrick,Samurai,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,20,1,NaN,NaN,NaN,NaN


In [ ]:
#create new DataFrame with only 'Name' and 'Level' columns
level_df = class_df[['Name', 'Level']]

#confimr new df
level_df.head(5)

In [6]:
# Get a list of unique owners
owners = df['owner'].unique()

# Create a DataFrame to store item counts by owner
item_count_df = pd.DataFrame({'owner': owners})

# Define the rarity types
rarities = ['Common', 'Uncommon', 'Rare', 'Very Rare', 'Legendary', 'Unique']

# Iterate through each rarity type
for rarity in rarities:
    # Create a dictionary to store rarity counts for each owner
    rarity_count = {}
    
    # Iterate through each owner
    for owner in owners:
        # Filter items for the current owner and rarity
        owner_items = df[(df['owner'] == owner) & (df['rarity'] == rarity)]
        
        # Count the number of items
        count = len(owner_items)
        
        # Store the count in the dictionary
        rarity_count[owner] = count
    
    # Add a column for the current rarity type
    item_count_df[rarity] = item_count_df['owner'].map(rarity_count)

# Fill NaN values with 0
item_count_df = item_count_df.fillna(0)

# Set 'Owner' as the index
item_count_df.set_index('owner', inplace=True)

# Display the resulting DataFrame
print(item_count_df)

                         Common  Uncommon  Rare  Very Rare  Legendary  Unique
owner                                                                        
LEE-ROY!                      0         4     3          5          3       0
Lynus                         1         3     3          4          2       0
Zalyria                       0         3     3          3          3       0
Yensid                        0         3     3          5          2       0
Superb                        0         3     3          4          2       0
...                         ...       ...   ...        ...        ...     ...
Ookgra                        0         1     0          0          0       0
Throgs                        0         1     0          0          0       0
Billetera de la Prisión       0         0     2          5          4       0
Mula de Articulo              0         3     4          3          0       0
Fangelsi Veski                0         0     1          7      

In [ ]:
# Calculate the sum of each row and assign it to a 'Total' column
item_count_df['Total'] = item_count_df.sum(axis=1)

# Display the updated DataFrame
print(item_count_df)

In [ ]:
# Perform an inner join on 'Name' in 'level_df' and 'Owner' in 'item_count_df'
result_df = item_count_df.merge(level_df, left_on='owner', right_on='Name', how='inner')

# Drop the 'Owner' column, as it's no longer needed
#result_df.drop(columns=['Owner'], inplace=True)

# Display the resulting DataFrame
print(result_df)

In [ ]:
# Get the 'Name' column
name_column = result_df.pop('Name')

# Insert the 'Name' column as the first column
result_df.insert(0, 'Name', name_column)

# Display the updated DataFrame
print(result_df)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Define the predictor variables (X) and response variable (y)
X = result_df[['Common', 'Uncommon', 'Rare', 'Very Rare', 'Legendary', 'Unique', 'Total']]
y = result_df['Level']

# Split the data into training and testing sets (optional but recommended)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

In [ ]:
# Define a function to collect user input and make predictions
def predict_level():
    print("Enter values for prediction:")
    
    # Collect user input for predictor variables
    common = float(input("Common: "))
    uncommon = float(input("Uncommon: "))
    rare = float(input("Rare: "))
    very_rare = float(input("Very Rare: "))
    legendary = float(input("Legendary: "))
    unique = float(input("Unique: "))
    total = float(input("Total: "))
    
    # Create a DataFrame with the user input
    user_input = pd.DataFrame({
        'Common': [common],
        'Uncommon': [uncommon],
        'Rare': [rare],
        'Very Rare': [very_rare],
        'Legendary': [legendary],
        'Unique': [unique],
        'Total': [total]
    })
    
    # Use the model to make predictions
    level_prediction = model.predict(user_input)
    
    # Round the prediction to the nearest integer
    rounded_prediction = round(level_prediction[0])
    
    # Display the rounded predicted level
    print(f"Predicted Level (Rounded): {rounded_prediction}")

# Call the function to collect input and make predictions
predict_level()